## Model 6: IF +FS2 (kernel)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../models/iforest_df_test_cleaned_pred_kt.csv',sep=';', index_col=0)

In [3]:
df['iforest_score'] =df['iforest_score']*-1

In [4]:
df.head()

,iforest_score,ind_var30,var36,var15,num_var4,num_var8_0,ind_var13_0,ind_var12_0,saldo_var30,ind_var39_0,...,var36_log,num_aport_var33_ult1_log,ind_var14_0_log,num_var4_log,imp_compra_var44_hace3_log,num_compra_var44_hace3_log,saldo_medio_var13_largo_hace3_log,ind_var43_recib_ult1_log,num_trasp_var11_ult1_log,saldo_var30_log
50595,0.304430,1.0,1.0,45.0,2.0,0.0,0.0,0.0,781.47,1.0,...,0.000000,0.010986,0.0,0.693147,0.080128,0.010986,0.050796,0.0,0.010986,6.661177
5252,0.314397,1.0,3.0,60.0,2.0,0.0,1.0,0.0,240003.00,1.0,...,1.098612,0.010986,0.0,0.693147,0.080128,0.010986,0.050796,0.0,0.010986,12.388407
21956,0.297477,1.0,3.0,27.0,1.0,0.0,0.0,0.0,60.00,1.0,...,1.098612,0.010986,0.0,0.000000,0.080128,0.010986,0.050796,0.0,0.010986,4.094345
42714,0.305171,1.0,1.0,38.0,3.0,0.0,0.0,0.0,293.70,1.0,...,0.000000,0.010986,0.0,1.098612,0.080128,0.010986,0.050796,0.0,0.010986,5.682559
18494,0.297388,1.0,3.0,23.0,1.0,0.0,0.0,0.0,3.00,1.0,...,1.098612,0.010986,0.0,0.000000,0.080128,0.010986,0.050796,0.0,0.010986,1.098612


prob_0	= probabilidade de estar satisfeito  
prob_1 = probabilidade de churn

In [5]:
def ks(data=None,target=None, prob=None):
    ## finding at: https://www.listendata.com/2019/07/KS-Statistics-Python.html
    data['target0'] = 1 - data[target]
    data['bucket']  = pd.qcut(data[prob].rank(method='first'), 5)
    # data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable.index = range(1,6)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    # print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [6]:
results_quintil = ks(df, target='TARGET', prob='iforest_score')
results_quintil

KS is 24.3% at decile 3


,min_prob,max_prob,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS
Decile,,,,,,,,,
1,0.301426,0.549349,121,2920,20.0,20.0,20.0,20.0,-0.1
2,0.298615,0.301426,196,2845,32.0,19.0,52.0,39.0,12.7
3,0.298295,0.298615,189,2851,31.0,20.0,83.0,59.0,24.3
4,0.297656,0.298295,52,2989,8.6,20.0,92.0,80.0,12.4
5,0.297380,0.297656,49,2992,8.1,20.0,100.0,100.0,0.0


In [7]:
from sklearn.metrics import roc_auc_score

print("Roc Auc: ",roc_auc_score(df['TARGET'], df['iforest_score']))
print("Gini: ",2*(roc_auc_score(df['TARGET'], df['iforest_score']))-1)

Roc Auc:  0.5990708185281917
Gini:  0.19814163705638332


In [8]:
from scipy.stats import ks_2samp
yhat = df['iforest_score']
y = df['TARGET']
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic
ks_stat(y, yhat)

0.2686318497211011

In [9]:
results_quintil.columns = ['min_prob','max_prob','Qtd de eventos (Target)','Qtd de não-eventos (Target)','% de eventos no Decil','% de não-eventos no Decil','% de evento acumulados','% de não-evento acumulados','KS']
results_quintil.to_csv("results_model6_iforest_fs2.csv",sep=';')